In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt

In [ ]:
# Reloadable libraries
import fd_design_parameter_solve, importlib
importlib.reload(fd_design_parameter_solve)
from fd_design_parameter_solve import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
# Regular Torus 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/200, 1/20]
MODEL_NAME = "regular_torus_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [ ]:
# # Single Linkage Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [0.01, 0.01]
# MODEL_PATH = osp.join('../../examples/' + 'single_linkage.obj')
# SUBDIVISION_RESOLUTION = 5

In [ ]:
linkage = elastic_rods.SurfaceAttractedLinkage(SURFACE_PATH, True, MODEL_PATH, SUBDIVISION_RESOLUTION, False)
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.updateSourceFrame()
linkage.set_design_parameter_config(True, True)
linkage.set_attraction_tgt_joint_weight(0)
linkage.set_holdClosestPointsFixed(False)

### test example 1

In [ ]:
# rk_vars = np.ones(linkage.numRestKappaVars()) 
# linkage.setRestKappaVars(rk_vars)
# assert(linkage.get_rest_kappa_smoothing() == 0)

### test example 2

In [ ]:
# for i in range(linkage.numSegments()):
#     small_rk_vars = np.ones(linkage.segment(i).rod.numRestKappaVars())
#     small_rk_vars[0] = 2
#     small_rk_vars[-1] = 2
#     linkage.segment(i).rod.setRestKappaVars(small_rk_vars)
# assert(linkage.get_rest_kappa_smoothing() == 180)

### test example 3

In [ ]:
# rk_vars = np.pi * np.random.normal(-1, 1, linkage.numRestKappaVars())
# assert(linkage.get_rest_kappa_smoothing() == 0)

### test example 4 (take longer)

In [ ]:
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
import py_newton_optimizer
design_opts = py_newton_optimizer.NewtonOptimizerOptions()
design_opts.niter = 5
design_opts.verbose = 10
with so(): elastic_rods.designParameter_solve(linkage, design_opts, 0, 0.1)
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)

### Compute equilibrium and visualize (not recommended for test example 1, 2 or 3)

In [ ]:
with so():
    opts = py_newton_optimizer.NewtonOptimizerOptions()
    opts.niter = 10
    opts.verbose = 10
    elastic_rods.compute_equilibrium(linkage, options = opts)

In [ ]:
view = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)
view.show()

### Define design parameter solve problem for finite difference validation

In [ ]:
linkage.attraction_weight = 1

In [ ]:
dp_problem = elastic_rods.DesignParameterProblem(linkage)

In [ ]:
dp_problem.set_regularization_weight(0)
dp_problem.set_smoothing_weight(0)
dp_problem.set_gamma(1)

In [ ]:
long_direction = np.random.normal(-1, 1, dp_problem.numVars())
gradient_convergence_plot(dp_problem, minStepSize=1e-9, maxStepSize=1e-1, direction = long_direction)

In [ ]:
hessian_convergence_plot(dp_problem, minStepSize=1e-11, maxStepSize=1e-1, direction = long_direction)